In [1]:
%load_ext lab_black

In [1]:
import pandas as pd
from nltk import tokenize
import boto3
import botocore
from datetime import datetime
import matplotlib.pylab as plt
import yfinance as yf
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from numba import jit, prange

In [2]:
# Load appropriate packages for sentiment analysis
from flair.models import TextClassifier
from flair.models import SequenceTagger
from flair.data import Sentence
from segtok.segmenter import split_single
import pandas as pd
import re


classifier = TextClassifier.load("en-sentiment")

2021-03-10 10:31:53,031 loading file C:\Users\Chris\.flair\models\sentiment-en-mix-distillbert_3.1.pt


In [6]:
nvidia_cleaned_df = pd.read_csv("../00_data/Cleaned_Data_10MAR.gz", compression="gzip")

In [7]:
nvidia_cleaned_df["VideoTitle"] = nvidia_cleaned_df["VideoTitle"].astype("str")
nvidia_cleaned_df["datePub"] = pd.to_datetime(
    nvidia_cleaned_df["datePub"], format="%Y-%m-%d %H:%M:%S", utc=True
)
nvidia_cleaned_df["searchedDate"] = pd.to_datetime(
    nvidia_cleaned_df["searchedDate"], format="%Y-%m-%d %H:%M:%S", utc=True
)
nvidia_cleaned_df["channelTitle"] = nvidia_cleaned_df["channelTitle"].astype("str")
nvidia_cleaned_df["viewCount"] = nvidia_cleaned_df["viewCount"].astype("int")
nvidia_cleaned_df["likeCount"] = nvidia_cleaned_df["likeCount"].astype("int")
nvidia_cleaned_df["dislikeCount"] = nvidia_cleaned_df["dislikeCount"].astype("int")
nvidia_cleaned_df["captionString"] = nvidia_cleaned_df["captionString"].astype("str")

In [11]:
nvidia_cleaned_df.dtypes

Unnamed: 0                     int64
Index                          int64
videoID                       object
datePub          datetime64[ns, UTC]
searchedDate     datetime64[ns, UTC]
VideoTitle                    object
channelTitle                  object
viewCount                      int32
likeCount                      int32
dislikeCount                   int32
captionString                 object
Date             datetime64[ns, UTC]
dtype: object

In [16]:
nvidia_cleaned_df.head()

,Unnamed: 0,Index,videoID,datePub,searchedDate,VideoTitle,channelTitle,viewCount,likeCount,dislikeCount,captionString,Date
0,0,0,y4fb_R5Ogw0,2020-10-01 05:16:02+00:00,2020-10-02 01:45:21.553469+00:00,"2x NVIDIA RTX 3090 SLI Benchmarks: 500FPS, 700...",Gamers Nexus,367472,15701,260,so this setup that you're seeing is what we ha...,2020-10-02
1,1,1,ALEXVtnNEwA,2020-09-01 17:15:57+00:00,2020-10-02 01:45:22.351801+00:00,NVIDIA GeForce Special Event Livestream,IGN,543115,16860,411,b welcome to my kitchen i hope all of you are ...,2020-10-02
2,2,2,E98hC9e__Xs,2020-09-01 16:42:07+00:00,2020-10-02 01:45:23.252012+00:00,NVIDIA GeForce RTX 30 Series | Official Launch...,NVIDIA GeForce,1484075,54741,1135,Welcome to my kitchen. I hope all of you are...,2020-10-02
3,3,3,nMns8t4OhI0,2020-09-30 23:00:06+00:00,2020-10-02 01:45:23.922084+00:00,NVIDIA Responds to RTX 3080 Crashes...,JayzTwoCents,561238,27766,885,nan,2020-10-02
4,4,4,AG_ZHi3tuyk,2020-09-16 12:59:59+00:00,2020-10-02 01:45:24.592244+00:00,"Nvidia, you PROMISED! - RTX 3080 Review",Linus Tech Tips,3391565,141852,3961,- The RTX 3080 is the most powerful GPU we've ...,2020-10-02


In [15]:
nvidia_cleaned_df["Date"] = pd.to_datetime(
    nvidia_cleaned_df["searchedDate"], format="%Y-%m-%d", utc=True
).dt.date

In [9]:
nvidia_cleaned_df["Date"] = nvidia_cleaned_df["searchedDate"].Timestamp.day_of_year

AttributeError: 'Series' object has no attribute 'Timestamp'

Vader Sentiment

In [ ]:
# load vader sentiment for comparing the uncleaned strings to the modified ones
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
nvidia_cleaned_df.columns

In [ ]:
print(f'Title to score = {nvidia_cleaned_df["captionString"][5]}')
score = sid.polarity_scores(str(nvidia_cleaned_df["captionString"][5]))
print(score)

In [ ]:
# utilize regular expressions to split large stings into sentences that are centered around the key words
import re

text = " allet wants to redefine the wallet with its compact frameand RFID blocking plates. Check out how they can helpkeep your wallet bulge down and use our offer code Linus to save 10% and getfree worldwide shipping. (upbeat music) Now, obviously I'm notready to hand Nvidia their crown until we seethe new cards for ourselves. But based on what they've shown, we are in for one heck of a ride. They announced well a lot andwe're gonna get to all of it, but the big one is, Ampere is real and Ampere is an absolute monster. As it turns out, the recentleaks were accurate in many if not most ways."


def search(target, text, context=6):
    # It's easier to use re.findall to split the string,
    # as we get rid of the punctuation
    words = re.findall(r"\w+", text)

    matches = (i for (i, w) in enumerate(words) if w.lower() == target)
    for index in matches:
        if index < context // 2:
            yield words[0 : context + 1]
        elif index > len(words) - context // 2 - 1:
            yield words[-(context + 1) :]
        else:
            yield words[index - context // 2 : index + context // 2 + 1]


print(list(search("nvidia", text, 10)))

In [ ]:
search("nvidia", text, 15)

In [ ]:
test_list_to_str = list(search("nvidia", text, 30))
test_list_to_str

In [ ]:
new_str = " ".join(test_list_to_str[0])

In [ ]:
score = sid.polarity_scores(new_str)
print(score)

### Flair Sentiment

In [4]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load("en-sentiment")
sentence = Sentence("Happy to be here")
classifier.predict(sentence)
# print sentence with predicted labels
print("Sentence above is: ", sentence.labels)

2021-03-31 16:22:31,677 loading file C:\Users\Chris\.flair\models\sentiment-en-mix-distillbert_4.pt
Sentence above is:  [POSITIVE (0.998)]


In [5]:
sentence.labels

[POSITIVE (0.998)]

In [6]:
sentence = Sentence("Happy to be here")
classifier.predict(sentence)